In [1]:
import cv2
import matplotlib.pyplot as plt
import sys

haar_cascade_face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


cap = cv2.VideoCapture(0)
alpha = 0
naruto = cv2.imread('naruto.png')

while(True):
    try:
        ret, frame = cap.read()
        frame_gray =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces_rects = haar_cascade_face.detectMultiScale(frame_gray, scaleFactor = 1.2, minNeighbors = 5);

        for (x, y, w, h) in faces_rects:
            #cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # crio um recorte do plano do frame onde vai ficar a máscara
            roi = frame[0:y+int(h/3), x:x+w]
            cv2.imshow('roi', roi)
           
            naruto_resize = cv2.resize(naruto, (roi.shape[1], roi.shape[0]))
            
            # crio a máscara e a inversa dela
            naruto_gray = cv2.cvtColor(naruto_resize,cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(naruto_gray, 10, 255, cv2.THRESH_BINARY)
            cv2.imshow('mask', mask)
            
            mask_inv = cv2.bitwise_not(mask)
            cv2.imshow('mask_inv', mask_inv)
            
            # removo a área que vai receber a imagem da minha área de recorte
            frame_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
            cv2.imshow('frame_bg', frame_bg)
            
            # pego só a parte que importa da imagem (excuindo o fundo)
            naruto_fg = cv2.bitwise_and(naruto_resize,naruto_resize,mask = mask)
            cv2.imshow('naruto_fg', naruto_fg)

            #coloco a imagem na aréa recortada e substituo na imagem original
            dst = cv2.add(frame_bg,naruto_fg)
            frame[0:y+int(h/3), x:x+w] = dst
    
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    except Exception as e:
        print(e)
        break
        
cap.release()
cv2.destroyAllWindows()